In [1]:
import boto3
import boto3
AWS_S3_BUCKET= ''
PATH_BRONZE=''
PATH_SILVER=''
LOCAL_PATH=''
s3 = boto3.client('s3')

In [2]:
s3.upload_file(f'{LOCAL_PATH}/paises.csv', AWS_S3_BUCKET, f'{PATH_BRONZE}/paises.csv')

In [3]:
import io
import pandas as pd
obj = s3.get_object(Bucket=AWS_S3_BUCKET, Key=f'{PATH_BRONZE}/paises.csv')
df = pd.read_csv(io.BytesIO(obj['Body'].read()))
df.head(5)


,name,nombre,iso
0,Andorra,Andorra,AD
1,United Arab Emirates,Emiratos Árabes Unidos,AE
2,Afghanistan,Afganistán,AF
3,Antigua and Barbuda,Antigua y Barbuda,AG
4,Anguilla,Anguila,AI


In [4]:
with io.BytesIO() as parquet_buffer:
    df.to_parquet(parquet_buffer, engine='auto', compression='snappy')

    response = s3.put_object(
        Bucket=AWS_S3_BUCKET, Key=f"{PATH_SILVER}paises.parquet.snappy", Body=parquet_buffer.getvalue()
    )

    status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

    if status == 200:
        print(f"Successful S3 put_object response. Status - {status}")
    else:
        print(f"Unsuccessful S3 put_object response. Status - {status}")

Successful S3 put_object response. Status - 200


In [5]:
def pd_read_s3_parquet(key, bucket, s3_client=None, **args):
    if s3_client is None:
        s3_client = boto3.client('s3')
    obj = s3_client.get_object(Bucket=bucket, Key=key)
    return pd.read_parquet(io.BytesIO(obj['Body'].read()), **args)

In [6]:
pd_read_s3_parquet(f"{PATH_SILVER}paises.parquet.snappy",AWS_S3_BUCKET,s3_client=s3)

,name,nombre,iso
0,Andorra,Andorra,AD
1,United Arab Emirates,Emiratos Árabes Unidos,AE
2,Afghanistan,Afganistán,AF
3,Antigua and Barbuda,Antigua y Barbuda,AG
4,Anguilla,Anguila,AI
...,...,...,...
233,Yugoslavia,Yugoslavia,YU
234,South Africa,Sudáfrica,ZA
235,Zambia,Zambia,ZM
236,Zaire,Zaire,ZR
